In [24]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
import numpy as np
# https://github.com/matplotlib/ipympl#installation for matplotlib images to appear in notebook
%matplotlib widget

In [95]:
x_op = [0.05, 0.1, 0.12]

In [102]:
n_pts = 25

theta = np.linspace(0, 2 * np.pi, n_pts)
phi   = np.linspace(0, np.pi, n_pts)
r = 0.1

X = r * np.outer(np.cos(theta), np.sin(phi)) + x_op[0]
Y = r * np.outer(np.sin(theta), np.sin(phi)) + x_op[1]
Z = r * np.outer(np.ones(np.size(theta)), np.cos(phi)) + x_op[2]

In [301]:
# https://gist.github.com/WetHat/1d6cd0f7309535311a539b42cccca89c
class Arrow3D(FancyArrowPatch):
    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)

def _arrow3D(ax, x, y, z, dx, dy, dz, muh_label, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)
    dd = np.array([x+dx, y+dy, z+dz])
    dd = dd / np.linalg.norm(dd)
    eps = 0.025
    ax.text(x+dx+eps*dd[0], y+dy+eps*dd[1], z+dz+eps*dd[2], muh_label, ha="center", va="center")


setattr(Axes3D, 'arrow3D', _arrow3D)

In [285]:
J = np.array([[1, 2, 5], [3, 4, 6]])
J

array([[1, 2, 5],
       [3, 4, 6]])

In [286]:
np.linalg.svd(J)[0] # u

array([[-0.56849697,  0.82268536],
       [-0.82268536, -0.56849697]])

In [287]:
sigmas = np.linalg.svd(J)[1] # singular values

In [288]:
sigmas = sigmas

In [289]:
u1, u2 = np.transpose(np.linalg.svd(J)[0])
u1

array([-0.56849697, -0.82268536])

In [290]:
np.dot(u1, u2)

0.0

In [291]:
np.linalg.svd(J)[2] # vt

array([[-0.32127519, -0.46846588, -0.82299573],
       [-0.68355838, -0.48673972,  0.54390476],
       [-0.65538554,  0.73730873, -0.16384638]])

In [292]:
v1, v2, v3 = np.linalg.svd(J)[2]
v1

array([-0.32127519, -0.46846588, -0.82299573])

In [293]:
np.dot(J, v1)

array([-5.3731856 , -7.77566347])

In [294]:
u1 * sigmas[0]

array([-5.3731856 , -7.77566347])

In [295]:
theta = np.linspace(0, 2*np.pi)

In [296]:
x = np.array([sigmas[0] * u1[0] * np.sin(t) + sigmas[1] * u2[0] * np.cos(t) for t in theta])
y = np.array([sigmas[0] * u1[1] * np.sin(t) + sigmas[1] * u2[1] * np.cos(t) for t in theta])
x

array([ 1.06248599,  0.36665527, -0.33519592, -1.0315432 , -1.71095257,
       -2.36226814, -2.97479534, -3.53847649, -4.04405594, -4.48323211,
       -4.84879373, -5.13473829, -5.33637059, -5.45037984, -5.474894  ,
       -5.40951055, -5.25530308, -5.01480369, -4.69196135, -4.29207714,
       -3.82171715, -3.28860466, -2.70149338, -2.07002365, -1.40456418,
       -0.71604182, -0.01576206,  0.6847765 ,  1.37407106,  2.04080341,
        2.67402582,  3.2633408 ,  3.79907181,  4.27242217,  4.67561948,
        5.00204325,  5.2463336 ,  5.4044793 ,  5.4738836 ,  5.45340688,
        5.34338537,  5.14562562,  4.86337485,  4.50126759,  4.06524964,
        3.5624804 ,  3.00121534,  2.3906704 ,  1.74087072,  1.06248599])

In [297]:
def sketch_vector(x0, x, label=None, dx_mag=0.1):
    plt.arrow(x0[0], x0[1], x[0], x[1], color="k", head_width=0.5, length_includes_head=True)
    dx = dx_mag * x / np.linalg.norm(x)
    plt.text(x0[0] + x[0] + dx[0], x0[1] + x[1] + dx[1], label, ha="center", va="center")

In [298]:
m_op = np.array([10, 14])
fig, ax = plt.subplots()
plt.scatter(m_op[0], m_op[1], label="$\mathbf{m}_{op}=\mathbf{u}_3$", color="r")
plt.plot(x+m_op[0], y+m_op[1])
ax.set(xlabel="$m_1$", ylabel="$m_2$", title="response space", xticks=[0], yticks=[0])
sketch_vector(m_op, sigmas[0] * u1, label="$\mathbf{u}_1$", dx_mag=0.75)
sketch_vector(m_op, sigmas[1] * u2, label="$\mathbf{u}_2$", dx_mag=0.75)

# plt.arrow(m_op[0], m_op[1], sigmas[0] * u1[0], sigmas[0] * u1[1], head_width=0.5, length_includes_head=True, color="k")
# plt.arrow(m_op[0], m_op[1], sigmas[1] * u2[0], sigmas[1] * u2[1], head_width=0.5, length_includes_head=True, color="k")
# plt.text(x0[0] + x[0] + dx_label, x0[1] + x[1] + dx_label, label, ha="center", va="center")
ax.set_aspect('equal', 'box')
plt.xlim(xmin=0)
plt.ylim(ymin=0)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [304]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X, Y, Z, alpha=0.5, rstride=1, cstride=1)
ax.set(xlabel="$x_1$", ylabel="$x_2$", zlabel="$x_3$", 
       xticks=[0], yticks=[0], zticks=[0],
      title="composition space")
ax.set_xlim(xmin=0)
ax.set_ylim(ymin=0)
ax.set_zlim(zmin=0)

ax.scatter(x_op[0], x_op[1], x_op[2], color="r", s=50, label="$x_{op}$")
for (v, muh_label) in zip([v1, v2, v3], ["$\mathbf{v}_1$", "$\mathbf{v}_2$", "$\mathbf{v}_3$"]):
    ax.arrow3D(x_op[0], x_op[1], x_op[2],
               r*v[0], r*v[1], r*v[2],
               mutation_scale=20,
               ec='k',
               fc='k', muh_label=muh_label)
# https://github.com/matplotlib/matplotlib/issues/17172
ax.set_box_aspect([ub - lb for lb, ub in (getattr(ax, f'get_{a}lim')() for a in 'xyz')])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [300]:
X

array([[ 5.00000000e-02,  6.30526192e-02,  7.58819045e-02,
         8.82683432e-02,  1.00000000e-01,  1.10876143e-01,
         1.20710678e-01,  1.29335334e-01,  1.36602540e-01,
         1.42387953e-01,  1.46592583e-01,  1.49144486e-01,
         1.50000000e-01,  1.49144486e-01,  1.46592583e-01,
         1.42387953e-01,  1.36602540e-01,  1.29335334e-01,
         1.20710678e-01,  1.10876143e-01,  1.00000000e-01,
         8.82683432e-02,  7.58819045e-02,  6.30526192e-02,
         5.00000000e-02],
       [ 5.00000000e-02,  6.26078620e-02,  7.50000000e-02,
         8.69643811e-02,  9.82962913e-02,  1.08801839e-01,
         1.18301270e-01,  1.26632048e-01,  1.33651630e-01,
         1.39239910e-01,  1.43301270e-01,  1.45766220e-01,
         1.46592583e-01,  1.45766220e-01,  1.43301270e-01,
         1.39239910e-01,  1.33651630e-01,  1.26632048e-01,
         1.18301270e-01,  1.08801839e-01,  9.82962913e-02,
         8.69643811e-02,  7.50000000e-02,  6.26078620e-02,
         5.00000000e-02],
    

In [ ]:
class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

####################################################
# This part is just for reference if
# you are interested where the data is
# coming from
# The plot is at the bottom
#####################################################

# Generate some example data
mu_vec1 = np.array([0,0,0])
cov_mat1 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class1_sample = np.random.multivariate_normal(mu_vec1, cov_mat1, 20)

mu_vec2 = np.array([1,1,1])
cov_mat2 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class2_sample = np.random.multivariate_normal(mu_vec2, cov_mat2, 20)

In [ ]:
# concatenate data for PCA
samples = np.concatenate((class1_sample, class2_sample), axis=0)

# mean values
mean_x = np.mean(samples[:,0])
mean_y = np.mean(samples[:,1])
mean_z = np.mean(samples[:,2])

#eigenvectors and eigenvalues
eig_val, eig_vec = np.linalg.eig(cov_mat1)

################################
#plotting eigenvectors
################################    

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')

ax.plot(samples[:,0], samples[:,1], samples[:,2], 'o', markersize=10, color='g', alpha=0.2)
ax.plot([mean_x], [mean_y], [mean_z], 'o', markersize=10, color='red', alpha=0.5)
for v in eig_vec:
    #ax.plot([mean_x,v[0]], [mean_y,v[1]], [mean_z,v[2]], color='red', alpha=0.8, lw=3)
    #I will replace this line with:
    a = Arrow3D([mean_x, v[0]], [mean_y, v[1]], 
                [mean_z, v[2]], mutation_scale=20, 
                lw=3, arrowstyle="-|>", color="r")
    ax.add_artist(a)
ax.set_xlabel('x_values')
ax.set_ylabel('y_values')
ax.set_zlabel('z_values')

plt.title('Eigenvectors')

plt.draw()
plt.show()